# Caso de uso de IA generativa: sumarizar o diálogo

Neste notebook veremos como realizar a tarefa de resumo de textos usando IA generativa. Você explorará como o texto de entrada afeta a saída do modelo e utilizará engenharia de prompt para instruir o modelo a realizar a tarefa. Ao comparar inferências de Zero shot, One shot e Few shot, você verá como a engenharia de prompt pode aprimorar a produção generativa de modelos de linguagem de larga escala (LLM).

# O que você vai encontrar aqui

- [ 1 - Dependências](#1)
- [ 2 - Sumarizando diálogo sem engenharia de prompt](#2)
- [ 3 - Sumarizando o diálogo com prompts de instrução](#3)
  - [ 3.1 - Inferência Zero Shot com um prompt de instrução](#3.1)
  - [ 3.2 - Inferência Zero Shot com o modelo de prompt do FLAN-T5](#3.2)
- [ 4 - Sumarizando o diálogo com inferências One Shot e Few Shot](#4)
  - [ 4.1 - Inferência One Shot](#4.1)
  - [ 4.2 - Inferência Few Shot](#4.2)
- [ 5 - Parâmetros de configuração](#5)


<a name='1'></a>
## 1 - Dependências

Instalando os pacotes necessários e usando PyTorche, Hugging Face transformers e datasets.

In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install -U \
    transformers==4.27.2 \
    datasets --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Carregando datasets, Large Language Model (LLM), tokenizer, and configurator.

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

/home/cquinta/aws-drops/projetos/llm-drops/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<a name='2'></a>
## 2 - Sumarizando diálogo sem engenharia de prompt

Neste caso de uso, você gerará um resumo de um diálogo com o Large Language Model (LLM) FLAN-T5 (Text-to-Text Transfer Transformer) pré-treinado do Hugging Face. A lista de modelos disponíveis no pacote Hugging Face `transformers` pode ser encontrada [aqui](https://huggingface.co/docs/transformers/index).

Vamos carregar alguns diálogos simples do dataset [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) do Hugging Face. Este conjunto de dados contém mais de 10.000 diálogos com os resumos correspondentes rotulados manualmente.

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Imprimindo alguns diálogos com seus resumos.

In [ ]:
example_indices = [10, 102]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print('DIÁLOGO DE ENTRADA:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('RESUMO HUMANO:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

Carregando o [modelo FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), criando uma instância da classe `AutoModelForSeq2SeqLM` com o método `.from_pretrained()`.

In [ ]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Para realizar codificação e decodificação, é necessário trabalhar com texto em formato tokenizado. **Tokenização** é o processo de divisão de textos em unidades menores que podem ser processadas pelos modelos LLMs.

Baixando o tokenizer para o modelo FLAN-T5, usando o método `AutoTokenizer.from_pretrained()`. O parâmetro `use_fast` ativa o tokenizer rápido. Você pode encontrar os parâmetros do tokenizer na [documentação](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoTokenizer).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

Testando a codificação do tokenizer e a decodificação de uma frase simples.

In [ ]:
sentence = "A Ana Cunha desenha melhor do que o Palla!"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )

print('SENTENÇA CODIFICADA:')
print(sentence_encoded["input_ids"][0])
print('\nSENTENÇA DECODIFICADA:')
print(sentence_decoded)

Agora é hora de explorar quão bem o LLM resume um diálogo sem qualquer engenharia de prompt. **Engenharia de prompt** é um ato humano de alterar o **prompt** (texto de entrada) para melhorar a resposta de uma determinada tarefa.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{dialogue}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}')
    print(dash_line)
    print(f'GERADO PELO MODELO - SEM ENGENHARIA DE PROMPT:\n{output}\n')

Você pode ver que as suposições do modelo fazem algum sentido, mas não parece haver certeza de qual tarefa ele deve realizar. Parece que ele apenas inferiu a próxima frase do diálogo. A engenharia de prompt pode ajudar aqui.

<a name='3'></a>
## 3 - Sumarizando o diálogo com um prompt de instrução

A engenharia de prompt é um conceito importante no uso de modelos fundacionais para geração de texto. Você pode conferir [este blog](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) da Amazon Science para obter uma introdução rápida à engenharia de prompt.

<a name='3.1'></a>
### 3.1 - Inferência Zero Shot com um prompt de instrução

Para instruir o modelo a executar uma tarefa – sumarizar um diálogo – você pode pegar o diálogo e convertê-lo em um prompt de instrução. Isso geralmente é chamado de **inferência de Zero-Shot**. Você pode conferir [este blog da AWS](https://aws.amazon.com/blogs/machine-learning/zero-shot-prompting-for-the-flan-t5-foundation-model-in-amazon-sagemaker-jumpstart/) para uma rápida descrição do que é o aprendizado Zero-Shot e por que ele é um conceito importante para o modelo LLM.

Envolvendo o diálogo em uma instrução descritiva:

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{prompt}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}')
    print(dash_line)    
    print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}\n')

Isto é muito melhor! Mas o modelo ainda não capta as nuances das conversas.

<a name='3.2'></a>
### 3.2 - Inferência Zero Shot com o modelo de prompt do FLAN-T5

Vamos usar um prompt um pouco diferente. O FLAN-T5 tem muitos modelos de prompt publicados para determinadas tarefas [aqui](https://github.com/google-research/FLAN/tree/main/flan/v2). No código a seguir, usaremos um dos [prompts FLAN-T5 pré-construídos](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py):

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
        
    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )

    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{prompt}')
    print(dash_line)
    print(f'RESUMO HUMANO:\n{summary}\n')
    print(dash_line)
    print(f'GERADO PELO MODELO - ZERO SHOT:\n{output}\n')

Observe que esse prompt do FLAN-T5 ajudou um pouco, mas a saída ainda não capta as nuances da conversa. Isso é o que tentaremos resolver com algumas inferências.

<a name='4'></a>
## 4 - Sumarizando o diálogo com inferências One-Shot e Few-Shot

**Inferências One-Shot e Few-Shot** são as práticas de fornecer a um LLM um ou mais exemplos completos de pares de prompt-resposta que correspondem à sua tarefa - antes do prompt real que você deseja concluir. Isso é chamado de “aprendizado em contexto” (in-context learning) e coloca seu modelo em um estado que compreende sua tarefa específica. Você pode ler mais sobre isso [neste blog do HuggingFace](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

<a name='4.1'></a>
### 4.1 - One-Shot

Vamos construir uma função que pega uma lista de `example_indices_full`, gera um prompt com exemplos completos e, no final, anexa o prompt que você deseja que o modelo complete (`example_index_to_summarize`). Nós usaremos o mesmo modelo de prompt FLAN-T5 da seção [3.2](#3.2).

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

Construindo um prompt para realizar inferência única (One-Shot):

In [ ]:
example_indices_full = [10]
example_index_to_summarize = 102

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

Utilizando este prompt para realizar a inferência One-Shot:

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - ONE SHOT:\n{output}')

<a name='4.2'></a>
### 4.2 - Few-Shot

Explorando a inferência Few-Shot adicionando mais dois pares completos de resumo de diálogo ao prompt.

In [ ]:
example_indices_full = [10, 80, 120]
example_index_to_summarize = 102

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

Utilizando o prompt para realizar inferência Few-Shot:

In [ ]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')
print(dash_line)
print(f'GERADO PELO MODELO - FEW SHOT:\n{output}')

Neste caso, a inferência Few-Shot não proporcionou muita melhoria em relação à inferência One-Shot. E qualquer coisa acima de 5 ou 6 exemplos normalmente também não ajuda muito. Além disso, é importante ter certeza de não exceder o comprimento do contexto de entrada do modelo que, neste caso, é de 512 tokens. Qualquer coisa acima do comprimento do contexto será ignorada.

No entanto, você pode ver que fornecer pelo menos um exemplo completo (one shot) fornece ao modelo mais informações e melhora qualitativamente a sumarização.

<a name='5'></a>
## 5 - Parâmetros de configuração

É possível alterar os parâmetros de configuração do método `generate()` para ver uma saída diferente do LLM. Até agora o único parâmetro alterado foi `max_new_tokens=50`, que define o número máximo de tokens a serem gerados. Uma lista completa de parâmetros disponíveis pode ser encontrada na [documentação do Hugging Face Generation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig).

Uma maneira conveniente de organizar os parâmetros de configuração é usar a classe `GenerationConfig`.

In [ ]:
generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=4.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'GERADO PELO MODELO - FEW SHOT:\n{output}')
print(dash_line)
print(f'RESUMO HUMANO:\n{summary}\n')

Comentários relacionados à escolha dos parâmetros na célula de código acima:
- Escolher `max_new_tokens=10` tornará o texto de saída muito curto, então o resumo do diálogo será cortado.
- Colocando `do_sample = True` e alterando o valor da temperatura você obtém mais "criatividade" na saída.

Como você pode ver, a engenharia de prompt pode ser muito útil nesse caso de uso, mas existem algumas limitações. Nos próximos episódios, vamos explorar como usar o ajuste fino para ajudar o LLM a entender um caso de uso específico com mais profundidade!